# Introduction to Failure Analysis using Batfish

*intro sketch*

Failures are bad. Networks are designed to be fault-tolerant. Batfish can help ensure that failures will not violate the availability and security requirements of the network.

In this notebook, we will show you how to perform failure analysis with Batfish.

In [1]:
# Import packages and load questions
%run startup.py

/Users/matt/.pyenv/versions/3.6.4/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/matt/src/pybatfish/pybatfish/client/commands.py:50: UserWarning: Pybatfish public API is being updated, note that API names and parameters will soon change.
  "Pybatfish public API is being updated, note that API names and parameters will soon change.")


## Setup: Initializing the Network and Snapshot

`SNAPSHOT_PATH` below can be updated to point to a custom snapshot directory. See the [Batfish instructions](https://github.com/batfish/batfish/wiki/Packaging-snapshots-for-analysis) for how to package data for analysis.

More example networks are available in the [networks](https://github.com/batfish/batfish/tree/master/networks) folder of the Batfish repository.

In [2]:
NETWORK_NAME = "example_network"
SNAPSHOT_NAME = "example_snapshot"

SNAPSHOT_PATH = "networks/example"

bf_set_network(NETWORK_NAME)
bf_init_snapshot(SNAPSHOT_PATH, name=SNAPSHOT_NAME, overwrite=True)

'example_snapshot'

The network snapshot that we initialized above is illustrated below. You can view or download the devices' configuration files [here](https://github.com/batfish/pybatfish/tree/master/jupyter_notebooks/networks/example).

![example-network](https://raw.githubusercontent.com/batfish/pybatfish/master/jupyter_notebooks/networks/example/example-network.png)

All of the information we will show you in this notebook is dynamically computed by Batfish based on the configuration files for the network devices.

---



# Seeding failures
Batfish makes failure analysis easy by allowing to seed various kinds of failures, and determine the effect of each failure on essential network properties like end-to-end reachability.

Failures are seeded using the  [`bf_fork_snapshot` command](https://pybatfish.readthedocs.io/en/latest/api.html?highlight=fork_snapshot#pybatfish.client.commands.bf_fork_snapshot). This creates a new snapshot of our network, which we can then compare against the original using several differential questions. The differential questions help us pinpoint what has changed as a result of each failure.

# Failure Scenario 1: Core Router
In this example, we want to ensure that reachability from `host1` and `host2` is not affected by the failure of either of the core routers in AS2 (`as2core1` and `as2core2`). We use `bf_fork_snaphot` with the `deactive_nodes` parameter to initialize a new snapshot for each failure scenario.

In [3]:
FAIL_AS2CORE1_SNAPSHOT_NAME = "fail_as2core1"
bf_fork_snapshot(SNAPSHOT_NAME, FAIL_AS2CORE1_SNAPSHOT_NAME, deactivate_nodes=["as2core1"], overwrite=True)

FAIL_AS2CORE2_SNAPSHOT_NAME = "fail_as2core2"
bf_fork_snapshot(SNAPSHOT_NAME, FAIL_AS2CORE2_SNAPSHOT_NAME, deactivate_nodes=["as2core2"], overwrite=True)

'fail_as2core2'

## Assessing impact on availability and security 
Next, we'll Batfish's [`differentialReachability` question](https://pybatfish.readthedocs.io/en/latest/questions.html#pybatfish.question.bfq.reducedReachability) to check if each failure had any impact on what is reachable from either host. We specify which snapshots we want to compare with the parameters `snaphost` and `reference_snapshot`. Usually `snapshot` will be some hypothetical snapshot (e.g. a failure scenario or a candidate change), and `reference_snapshot` will be a real-world or a "last-known good" version. First, let's compare the failure scenario for `as2core1` against our real-world snapshot:

In [4]:
pathConstraints = PathConstraints(startLocation="host.")
answer = bfq.differentialReachability(pathConstraints=pathConstraints).answer(
    snapshot=FAIL_AS2CORE1_SNAPSHOT_NAME, 
    reference_snapshot=SNAPSHOT_NAME)
display_html(answer.frame())

,flow,Base_traces,Delta_traces


Great! The `reducedReachability` question does an *exhaustive search* over all possible flows, so the lack of results **guarantees** that the failure of `as2core1` will have no impact on reachability from either host. Now let's repeat the same check for the `as2core2` failure scenario:

In [5]:
answer = bfq.differentialReachability(pathConstraints=pathConstraints).answer(
    snapshot=FAIL_AS2CORE2_SNAPSHOT_NAME, 
    reference_snapshot=SNAPSHOT_NAME)
display_html(answer.frame())

,flow,Base_traces,Delta_traces
0,Src IP: 2.128.0.101Src Port: 0Dst IP: 1.0.1.0Dst Port: 23IP Protocol: TCPStart Location: host1,"DENIED_IN{blocktelnet}{deny tcp any any eq telnet}1 host1:eth0 → as2dept1:GigabitEthernet2/0Route(s): StaticRoute<0.0.0.0/0,nhip:2.128.0.1,nhint:eth0>_fnhip:2.128.0.12 as2dept1:GigabitEthernet0/0 → as2dist1:GigabitEthernet2/0Route(s): BgpRoute<1.0.1.0/24,nhip:2.34.101.3,nhint:dynamic>_fnhip:2.34.101.33 as2dist1:GigabitEthernet0/0 → as2core1:GigabitEthernet2/0Route(s): BgpRoute<1.0.1.0/24,nhip:10.12.11.1,nhint:dynamic>_fnhip:2.23.11.2DENIED_IN{blocktelnet}{deny tcp any any eq telnet}1 host1:eth0 → as2dept1:GigabitEthernet2/0Route(s): StaticRoute<0.0.0.0/0,nhip:2.128.0.1,nhint:eth0>_fnhip:2.128.0.12 as2dept1:GigabitEthernet1/0 → as2dist2:GigabitEthernet2/0Route(s): BgpRoute<1.0.1.0/24,nhip:2.34.201.3,nhint:dynamic>_fnhip:2.34.201.33 as2dist2:GigabitEthernet1/0 → as2core1:GigabitEthernet3/0Route(s): BgpRoute<1.0.1.0/24,nhip:10.12.11.1,nhint:dynamic>_fnhip:2.23.12.2","DENIED_IN{blocktelnet}{deny tcp any any eq telnet}1 host1:eth0 → as2dept1:GigabitEthernet2/0Route(s): StaticRoute<0.0.0.0/0,nhip:2.128.0.1,nhint:eth0>_fnhip:2.128.0.12 as2dept1:GigabitEthernet0/0 → as2dist1:GigabitEthernet2/0Route(s): BgpRoute<1.0.1.0/24,nhip:2.34.101.3,nhint:dynamic>_fnhip:2.34.101.33 as2dist1:GigabitEthernet0/0 → as2core1:GigabitEthernet2/0Route(s): BgpRoute<1.0.1.0/24,nhip:10.12.11.1,nhint:dynamic>_fnhip:2.23.11.2EXITS_NETWORK1 host1:eth0 → as2dept1:GigabitEthernet2/0Route(s): StaticRoute<0.0.0.0/0,nhip:2.128.0.1,nhint:eth0>_fnhip:2.128.0.12 as2dept1:GigabitEthernet0/0 → as2dist1:GigabitEthernet2/0Route(s): BgpRoute<1.0.1.0/24,nhip:2.34.101.3,nhint:dynamic>_fnhip:2.34.101.33 as2dist1:GigabitEthernet1/0 → as2core2:GigabitEthernet3/0Route(s): BgpRoute<1.0.1.0/24,nhip:10.12.11.1,nhint:dynamic>_fnhip:2.23.21.24 as2core2:GigabitEthernet1/0 → as2border1:GigabitEthernet2/0Route(s): BgpRoute<1.0.1.0/24,nhip:10.12.11.1,nhint:dynamic>_fnhip:2.12.12.15 as2border1:GigabitEthernet0/0 → as1border1:GigabitEthernet1/0Route(s): BgpRoute<1.0.1.0/24,nhip:10.12.11.1,nhint:dynamic>_fnhip:10.12.11.16 as1border1:GigabitEthernet0/0 → (none):null_interfaceRoute(s): ConnectedRoute<1.0.1.0/24,nhip:AUTO/NONE(-1l),nhint:GigabitEthernet0/0>_fnhip:nullEXITS_NETWORK1 host1:eth0 → as2dept1:GigabitEthernet2/0Route(s): StaticRoute<0.0.0.0/0,nhip:2.128.0.1,nhint:eth0>_fnhip:2.128.0.12 as2dept1:GigabitEthernet1/0 → as2dist2:GigabitEthernet2/0Route(s): BgpRoute<1.0.1.0/24,nhip:2.34.201.3,nhint:dynamic>_fnhip:2.34.201.33 as2dist2:GigabitEthernet0/0 → as2core2:GigabitEthernet2/0Route(s): BgpRoute<1.0.1.0/24,nhip:10.12.11.1,nhint:dynamic>_fnhip:2.23.22.24 as2core2:GigabitEthernet1/0 → as2border1:GigabitEthernet2/0Route(s): BgpRoute<1.0.1.0/24,nhip:10.12.11.1,nhint:dynamic>_fnhip:2.12.12.15 as2border1:GigabitEthernet0/0 → as1border1:GigabitEthernet1/0Route(s): BgpRoute<1.0.1.0/24,nhip:10.12.11.1,nhint:dynamic>_fnhip:10.12.11.16 as1border1:GigabitEthernet0/0 → (none):null_interfaceRoute(s): ConnectedRoute<1.0.1.0/24,nhip:AUTO/NONE(-1l),nhint:GigabitEthernet0/0>_fnhip:nullDENIED_IN{blocktelnet}{deny tcp any any eq telnet}1 host1:eth0 → as2dept1:GigabitEthernet2/0Route(s): StaticRoute<0.0.0.0/0,nhip:2.128.0.1,nhint:eth0>_fnhip:2.128.0.12 as2dept1:GigabitEthernet1/0 → as2dist2:GigabitEthernet2/0Route(s): BgpRoute<1.0.1.0/24,nhip:2.34.201.3,nhint:dynamic>_fnhip:2.34.201.33 as2dist2:GigabitEthernet1/0 → as2core1:GigabitEthernet3/0Route(s): BgpRoute<1.0.1.0/24,nhip:10.12.11.1,nhint:dynamic>_fnhip:2.23.12.2"
1,Src IP: 2.128.1.101Src Port: 0Dst IP: 1.0.1.0Dst Port: 23IP Protocol: TCPStart Location: host2,"DENIED_IN{blocktelnet}{deny tcp any any eq telnet}1 host2:eth0 → as2dept1:GigabitEthernet3/0Route(s): StaticRoute<0.0.0.0/0,nhip:2.128.1.1,nhint:eth0>_fnhip:2.128.1.12 as2dept1:GigabitEthernet0/0 → as2dist1:GigabitEthernet2/0Route(s): BgpRoute<1.0.1.0/24,nhip:2.34.101.3,nhint:dynamic>_fnhip:2.34.101.33 as2dist1:GigabitEthernet0/0 → as2core1:GigabitEthernet2/0Route(s): B

In this case we see that the failure causes a loss of availability from both hosts. Batfish returns two flows, one starting at each host. For each flow we can see all possible traces on each snapshot. Both flows are telnet (port `23`) traffic to `1.0.1.0`. In each snapshot, that traffic is dropped by `as2core1` by the ingress filter `blocktelnet`. However, in the real-world snapshot there is another path that goes through `as2core2` instead of `as2core1`, and the traffic isn't dropped along that path.

In general, flows that always dropped in the failure scenario demonstrate a loss of availability. Similarly, flows that are always dropped in the real-world or last-known-good snapshot demonstrate a loss of security. 

<small>Note: these flows demonstrate a *multipath inconsitency* in the real-world snapshot: a single flow that can be either delivered or dropped depending upon the path it takes. These issues are often an indication that the network is not robust to failures, as we see in this example. Batfish can automatically detect multipath inconsistencies via its [`multipathConsistency` question](https://pybatfish.readthedocs.io/en/latest/questions.html#pybatfish.question.bfq.multipathConsistency).</small>

# Failure Scenario 2: Border Router
In this scenario we analyze changes in reachability to `host1` or `host2` in the presence of `AS2` border router failures.

**TODO** given the topology, are we willing to accept some loss of availability if a border router fails? May want to skip the `as2border2` failure if we don't care about that loss of availability. Also, may want to add a parameter to `differentialReachability` to return only increased or decreased differences (then we could write wrapper question templates, e.g. `reducedAvailability` and `reducedSecurity`).

We are definitely not willing to accept loss of security, which this example demonstrates.

In [6]:
FAIL_AS2BORDER1_SNAPSHOT_NAME = "fail_as2border1"
bf_fork_snapshot(SNAPSHOT_NAME, FAIL_AS2BORDER1_SNAPSHOT_NAME, deactivate_nodes=["as2border1"], overwrite=True)

FAIL_AS2BORDER2_SNAPSHOT_NAME = "fail_as2border2"
bf_fork_snapshot(SNAPSHOT_NAME, FAIL_AS2BORDER2_SNAPSHOT_NAME, deactivate_nodes=["as2border2"], overwrite=True)

'fail_as2border2'

In [7]:
pathConstraints = PathConstraints(endLocation="host.")
answer = bfq.differentialReachability(pathConstraints=pathConstraints).answer(
    snapshot=FAIL_AS2BORDER1_SNAPSHOT_NAME, 
    reference_snapshot=SNAPSHOT_NAME)
display_html(answer.frame())

flow  \
0  start=as1border1 [1.0.1.1:0->2.128.1.101:22 TCP]   
1  start=as1border2 [1.0.2.1:0->2.128.1.101:22 TCP]   
2  start=as1core1 [1.0.1.2:0->2.128.1.101:22 TCP]     

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  

This example shows that the failure of `as2border1` leads to *increased* reachability from `AS1` to the hosts within `AS2`. In other words, there is a **loss of security** because the ingress ACL `OUTSIDE_TO_INSIDE` on `as2border1` is bypassed in the failure scenario when the traffic gets routed through `AS3`. In particular, SSH traffic is permitted to enter `as2border2` but blocked by `OUTSIDE_TO_INSIDE` when entering `as2border1`.

In [8]:
pathConstraints = PathConstraints(endLocation="host.")
answer = bfq.differentialReachability(pathConstraints=pathConstraints).answer(
    snapshot=FAIL_AS2BORDER2_SNAPSHOT_NAME, 
    reference_snapshot=SNAPSHOT_NAME)
display_html(answer.frame())

flow  \
0  start=as3border1 [3.0.1.1:0->2.128.1.101:22 TCP]   
1  start=as3border2 [3.0.2.1:0->2.128.1.101:22 TCP]   
2  start=as3core1 [3.0.1.2:0->2.128.1.101:22 TCP]     

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  Base_traces  \
0  [(as3border1:GigabitEthernet0/0 -> as3core1:GigabitEthernet1/0\n    Route(s):\n    BgpRoute<2.128.0.0/16,nhip:10.13.22.1,nhint:dynamic>_fnhip:3.0.1.2, as3core1:GigabitEthernet0/0 -> as3border2:GigabitEthernet1/0\n    Route(s):\n    BgpRoute<2.128.0.0/16,nhip:10.13.22.1,nhint:dynamic>_fnhip:3.0.2.1, as3border2:GigabitEthernet0/0 -> as1border2:GigabitEthernet0/0\n    Route(s):\n    BgpRoute<2.128.0.0/16,nhip:10.13.22.1,nhint:dynamic>_fnhip:10.13.22.1, as1border2:GigabitEthernet1/0 -> as1core1:GigabitEthernet0/0\n    Route(s):\n    BgpRoute<2.128.0.0/16,nhip:10.12.11.2,nhint:dynamic>_fnhip:1.0.2.2, as1core1:GigabitEthernet1/0 -> as1border1:GigabitEthernet0/0\n    Route(s):\n    BgpRoute<2.128.0.0/16,nhip:10.12.11.2,nhint:dynamic>_fnhip:1.0.1.1, as1border1:GigabitEthernet1/0 -> as2border1:GigabitEthernet0/0\n    Route(s):\n    BgpRoute<2.128.0.0/16,nhip:10.12.11.2,nhint:dynamic>_fnhip:10.12.11.2)]   
1  [(as3border2:GigabitEthernet0/0 -> as1border2:GigabitEthernet0/0\n    Route(s):\n    BgpRoute<2.128.0.0/16,nhip:10.13.22.1,nhint:dynamic>_fnhip:10.13.22.1, as1border2:GigabitEthernet1/0 -> as1core1:GigabitEthernet0/0\n    Route(s):\n    BgpRoute<2.128.0.0/16,nhip:10.12.11.2,nhint:dynamic>_fnhip:1.0.2.2, as1core1:GigabitEthernet1/0 -> as1border1:GigabitEthernet0/0\n    Route(s):\n    BgpRoute<2.128.0.0/16,nhip:10.12.11.2,nhint:dynamic>_fnhip:1.0.1.1, as1border1:GigabitEthernet1/0 -> as2border1:GigabitEthernet0/0\n    Route(s):\n    BgpRoute<2.128.0.0/16,nhip:10.12.11.2,nhint:dynamic>_fnhip:10.12.11.2)]                                                                                                                                                                                                                                                                                                             
2  [(as3core1:GigabitEthernet0/0 -> as3border2:GigabitEthernet1/0\n    Route(s):\n    BgpRoute<2.128.0.0/16,nhip:10.13.22.1,nhint:dynamic>_fnhip:3.0.2.1, as3border2:GigabitEthernet0/0 -> as1border2:GigabitEthernet0/0\n    Route(s):\n    BgpRoute<2.128.0.0/16,nhip:10.13.22.1,nhint:dynamic>_fnhip:10.13.22.1, as1border2:GigabitEthernet1/0 -> as1core1:GigabitEthernet0/0\n    Route(s):\n    BgpRoute<2.128.0.0/16,nhip:10.12.11.2,nhint:dynamic>_fnhip:1.0.2.2, as1core1:GigabitEthernet1/0 -> as1border1:GigabitEthernet0/0\n    Route(s):\n    BgpRoute<2.128.0.0/16,nhip:10.12.11.2,nhint:dynamic>_fnhip:1.0.1.1, as1border1:GigabitEthernet1/0 -> as2border1:GigabitEthernet0/0\n    Route(s):\n    BgpRoute<2.128.0.0/16,nhip:10.12.11.2,nhint:dynamic>_fnhip:10.12.11.2)]                                                                                                                                                        

                                                                                                                                                                             

This example shows that the failure of `as2border2` leads to *decreased* reachability from `AS3` to the hosts within `AS2`. In other words, there is a **loss of availability** because the traffic gets routed through `AS1` and enters `AS2` through `as2border1` instead of `as2border2`. The ingress ACL `OUTSIDE_TO_INSIDE` on `as2border1` blocks the SSH traffic that would have been permitted by `as2border2`.

## Wrap-up

This concludes the notebook. To recap, we showed how to perform failure analysis in two simple steps: 

1. Create new snapshots for failure scenarios using `bf_fork_snapshot`
2. Search for losses of availability or security using `differentialReachability` question.

We hope you found this notebook useful and informative. Future notebooks will dive into more advanced topics ensuring planned configuration changes do not have unintended consequences. Stay tuned!

### Want to know more? 

Reach out to us through [Slack](https://join.slack.com/t/batfish-org/shared_invite/enQtMzA0Nzg2OTAzNzQ1LTUxOTJlY2YyNTVlNGQ3MTJkOTIwZTU2YjY3YzRjZWFiYzE4ODE5ODZiNjA4NGI5NTJhZmU2ZTllOTMwZDhjMzA) or [Github](https://github.com/batfish/batfish) to learn more, or send feedback.